In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def rz(name: str) -> pd.DataFrame:
    """ Тестова функція. 
    name: ім'я судді у форматі : Прізвище Ім'я По-батькові
    """

    url = 'http://blood.in.ua/declaration/'
    response = requests.get(url + name.replace(' ', '-').lower()) # url = 'http://blood.in.ua/declaration/прізвище-ім'я-побатькові
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    
    # отримую все, крім посади
    dfs = pd.read_html(soup.prettify())
    df = dfs[0] # вміст таблиці в першому елементі
    df.columns = ['ПІБ', 'Місце роботи', 'Початок роботи', 'Кінець роботи']
    
    # посада за `em` тегом
    df['Посада'] = [em.text for em in soup.find_all('em')]
    
    # оскільки спочатку колонка місця роботи містить як саме місце роботи, так і посаду,
    # прибираю з неї збіг із колонкою посади
    df['Місце роботи'] = df['Місце роботи'].str.replace('|'.join(df['Посада']), '')
    
    # шукаю індекс, за якого у колонці місця роботи вказана посада
    idx = df.loc[df['Місце роботи'].str.strip().eq(df['Посада'].str.strip())].index
    
    # # замінити місцем роботи з колонки вгорі
    # df.loc[idx,'Місце роботи'] = df.loc[idx-1, :]['Місце роботи'].values 
    # # або порожнім значенням
    df.loc[idx,'Місце роботи'] = ''
    
    # замінити друге підряд однакове ПІБ порожнім значенням
    df.loc[df['ПІБ'].shift(1) == df['ПІБ'], 'ПІБ'] = ''
    # до тих ПІБ, що лишились, додати символ em dash в кінець 
    df.loc[df['ПІБ'].ne(''), 'ПІБ'] = df['ПІБ'] + ' —'
    
    # об'єднати дати початку та кінця роботи в одну колонку
    df['Період роботи'] = df['Початок роботи'].str.cat(df['Кінець роботи'], sep = '-')
    # з об'єднаної колонки отримали тільки роки початку та кінця, об'єднати та дописати ' рр.'
    df['Період роботи'] = df['Період роботи'].str.findall('(\d{4})').apply('—'.join) + ' рр.'
    
    return df[['ПІБ', 'Місце роботи', 'Посада', 'Період роботи']]

In [4]:
df = rz('Головенко Ольга Степанівна')

In [5]:
df

,ПІБ,Місце роботи,Посада,Період роботи
0,Головенко Андрій Васильович (племінник) —,Оперативне управління ДПІ м.Івано-Франківськ Г...,cтарший оперуповноважений відділу оперативного...,2015—2016 рр.
1,,Оперативне управління ГУ ДФС в Івано-Франківсь...,cтарший оперуповноважений з особливо важливих ...,2016—2018 рр.
2,Марусяк Олександр Любомирович (племінник) —,Державна Екологічна Інспекція в Івано-Франківс...,головний спеціаліст відділу екологічного та ра...,2012—2018 рр.
3,Марусяк Мар'ян Любомирович (племінник) —,Долинське відділення поліції Калуського відділ...,поліцейський сектору реагування патрульної пол...,2017—2017 рр.
4,,,інспектор з ювенальної превенції сектору преве...,2017—2018 рр.
